### Import libraries

In [1]:
%%capture
!pip install lightgbm --no-warn-script-location
!pip install lazypredict --no-warn-script-location

# import Lazy Predict classification library
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from sklearn.model_selection import train_test_split

# referenced code from [1] - [8] in https://kapernikov.com/tutorial-image-classification-with-scikit-learn/
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import os

### Prepare the data set

In [5]:
import joblib
from skimage.io import imread
from skimage.transform import resize

def resize_all(src, pklname, include, width=150, height=None):
    """
    load images from path, resize them and write them as arrays to a dictionary, 
    together with labels and metadata. The dictionary is written to a pickle file 
    named '{pklname}_{width}x{height}px.pkl'.
     
    Parameter
    ---------
    src: str
        path to data
    pklname: str
        path to output file
    width: int
        target width of the image in pixels
    include: set[str]
        set containing str
    """
     
    height = height if height is not None else width
     
    data = dict()
    data['description'] = 'resized ({0}x{1})western blot images in rgb'.format(int(width), int(height))
    data['label'] = []
    data['filename'] = []
    data['data'] = []   
     
    pklname = f"{pklname}_{width}x{height}px.pkl"
 
    # read all images in PATH, resize and write to DESTINATION_PATH
    for subdir in os.listdir(src):
        if subdir in include:
            print(subdir)
            current_path = os.path.join(src, subdir)
 
            for file in os.listdir(current_path):
                if file[-3:] in {'jpg', 'png'}:
                    im = imread(os.path.join(current_path, file))
                    # [:,:,::-1]
                    im = resize(im, (width, height))
                    data['label'].append(subdir[:-4])
                    data['filename'].append(file)
                    data['data'].append(im)
 
        joblib.dump(data, pklname)

In [3]:
data_path = fr'{os.getenv("HOME")}\Users\athenaz2\Documents\Western-Blot-Data\Images\aTub_Lerit_Lab_2022-07-12_13h11m29s(Chemiluminescence)_mani.png'
print(data_path[-8:-4])

mani
